<a href="https://colab.research.google.com/github/jeffersonramelo/Paper-pos-doc/blob/main/R2_dentro_e_fora_da_amostra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd

# Caminho para o arquivo XLSX
caminho_arquivo = '/content/Jefferson - Dados3 - Copia.xlsx'

# Lê o arquivo XLSX
dados = pd.read_excel(caminho_arquivo)

In [ ]:
# Supondo que 'dados' seja o seu DataFrame

# Converter a coluna 't' para o tipo de dado datetime, se ainda não estiver no formato adequado
dados['t'] = pd.to_datetime(dados['t'], format='%Y')

# Definir as datas de início e fim para o intervalo desejado
data_inicio = pd.to_datetime('2017-01-01')
data_fim = pd.to_datetime('2022-12-31')

# Filtrar os dados para extrair apenas as linhas no intervalo de 2010 a 2013
dados = dados.query('@data_inicio <= t <= @data_fim')

# Agora, 'dados_2010_2013' contém apenas as linhas com valores de 't' no intervalo de 2010 a 2013.


In [ ]:
dados

excluir setor financeiro e empresas com PL negativo, numeros faltante e transformar eventuais caracteres em números

In [11]:
# Agora, vamos remover as linhas onde 'd9' seja igual a 1
dados = dados[dados['d11'] != 1]
# Converte a coluna 'PL2010' para números (float) e remove as linhas onde 'PL2010' seja negativo
dados['PL2010'] = pd.to_numeric(dados['PL2010'], errors='coerce')  # 'coerce' trata strings não numéricas como NaN
dados = dados[dados['PL2010'] > 0]

# Agora, vamos remover as linhas onde 'VM30Abril11' seja igual a '-'
dados = dados[dados['VM30Abril2011'] != '-']
dados = dados[dados['PL2010'] != '-']
dados = dados[dados['LL2010'] != '-']
dados = dados[dados['AT2009'] != '-']
dados = dados[dados['AT2009'] != '0']

#transforma as variáveis em números
dados['AT2009'] = pd.to_numeric(dados['AT2009'], errors='coerce')
dados['LL2010'] = pd.to_numeric(dados['LL2010'], errors='coerce')
dados['VM30Abril2011'] = pd.to_numeric(dados['VM30Abril2011'], errors='coerce')
dados['PL2010'] = pd.to_numeric(dados['PL2010'], errors='coerce')

# Lista de variáveis que você deseja escalonar
variaveis = ['VM30Abril2011', 'PL2010', 'LL2010', 'AT2009']

# Verifique o tipo de dados de cada coluna
for coluna in variaveis:
    print(f"Tipo de dados da coluna {coluna}: {dados[coluna].dtype}")


Tipo de dados da coluna VM30Abril2011: float64
Tipo de dados da coluna PL2010: float64
Tipo de dados da coluna LL2010: float64
Tipo de dados da coluna AT2009: float64


<ipython-input-11-28e051817eca>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['PL2010'] = pd.to_numeric(dados['PL2010'], errors='coerce')  # 'coerce' trata strings não numéricas como NaN


ESCALONAMENTO DAS VARIÁVEIS

In [ ]:
variaveis = ['VM30Abril2011', 'PL2010', 'LL2010']

for coluna in variaveis:
    dados[coluna] = dados[coluna] / dados['AT2009']


WINSORIZAÇÃO: tratar outliers: A winsorização ao nível de 1% envolve substituir os 1% dos valores mais extremos por valores no limite do intervalo aceitável, ou seja, os 1% menores e os 1% maiores dos valores.

In [12]:
import pandas as pd
import numpy as np
from scipy.stats import mstats

# Supondo que seu DataFrame se chame "dados"

# Lista de variáveis para winsorizar
variaveis_winsorizar = ['VM30Abril2011', 'PL2010', 'LL2010', 'AT2009']

# Converter as colunas para números, tratando não numéricos como NaN
for coluna in variaveis_winsorizar:
    dados[coluna] = pd.to_numeric(dados[coluna], errors='coerce')

# Aplicar winsorização de 1% nas variáveis
for coluna in variaveis_winsorizar:
    dados[coluna] = mstats.winsorize(dados[coluna], limits=[0.01, 0.01])

# Agora, as variáveis especificadas foram winsorizadas


MQO_SIMPLES

In [14]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

# Suponha que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Certifique-se de que os tipos de dados sejam numéricos
X = X.astype(float)
y = y.astype(float)

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
r2_in_sample_scores = []
r2_out_of_sample_scores = []

# Treinar e avaliar o modelo usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Crie o modelo de regressão linear
    modelo = sm.OLS(y_train, X_train).fit()

    # Calcule o R^2 dentro da amostra
    r2_in_sample = modelo.rsquared
    r2_in_sample_scores.append(r2_in_sample)

    # Faça previsões no conjunto de teste
    y_pred = modelo.predict(X_test)

    # Calcule o R^2 fora da amostra
    r2_out_of_sample = r2_score(y_test, y_pred)
    r2_out_of_sample_scores.append(r2_out_of_sample)

# Média dos R^2 dentro e fora da amostra
media_r2_in_sample = np.mean(r2_in_sample_scores)
media_r2_out_of_sample = np.mean(r2_out_of_sample_scores)

print("Média do R^2 dentro da amostra:", media_r2_in_sample)
print("Média do R^2 fora da amostra:", media_r2_out_of_sample)


Média do R^2 dentro da amostra: 0.9477442691461508
Média do R^2 fora da amostra: 0.6677571131050917


MQP - MODELO SIMPLES

In [15]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

# Suponha que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Certifique-se de que os tipos de dados sejam numéricos
X = X.astype(float)
y = y.astype(float)

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
r2_in_sample_scores = []
r2_out_of_sample_scores = []

# Treinar e avaliar o modelo robusto usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Crie o modelo de regressão robusta
    modelo_robusto = sm.RLM(y_train, X_train, M=sm.robust.norms.HuberT()).fit()

    # Calcule o R^2 dentro da amostra
    r2_in_sample = r2_score(y_train, modelo_robusto.fittedvalues)
    r2_in_sample_scores.append(r2_in_sample)

    # Faça previsões no conjunto de teste
    y_pred = modelo_robusto.predict(X_test)

    # Calcule o R^2 fora da amostra
    r2_out_of_sample = r2_score(y_test, y_pred)
    r2_out_of_sample_scores.append(r2_out_of_sample)

# Média dos R^2 dentro e fora da amostra
media_r2_in_sample = np.mean(r2_in_sample_scores)
media_r2_out_of_sample = np.mean(r2_out_of_sample_scores)

print("Média do R^2 dentro da amostra:", media_r2_in_sample)
print("Média do R^2 fora da amostra:", media_r2_out_of_sample)


Média do R^2 dentro da amostra: 0.9460100113664194
Média do R^2 fora da amostra: 0.6628894987967024


RB_Simples (LINEAR)

In [17]:
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import KFold
from sklearn.metrics import median_absolute_error, r2_score
import numpy as np

# Suponha que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_in_sample_scores = []
r2_out_of_sample_scores = []

# Treinar e avaliar o modelo usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo Bayesian Ridge
    modelo_bayesiano = BayesianRidge()

    modelo_bayesiano.fit(X_train, y_train)
    y_pred = modelo_bayesiano.predict(X_test)

    # Calcular o erro mediano absoluto (Median Absolute Error)
    medae = median_absolute_error(y_test, y_pred)
    medae_scores.append(medae)

    # Calcular o R^2 dentro da amostra
    y_train_pred = modelo_bayesiano.predict(X_train)
    r2_in_sample = r2_score(y_train, y_train_pred)
    r2_in_sample_scores.append(r2_in_sample)

    # Calcular o R^2 fora da amostra
    r2_out_of_sample = r2_score(y_test, y_pred)
    r2_out_of_sample_scores.append(r2_out_of_sample)

# Média das métricas de avaliação em todas as dobras
media_medae = np.mean(medae_scores)
media_r2_in_sample = np.mean(r2_in_sample_scores)
media_r2_out_of_sample = np.mean(r2_out_of_sample_scores)

print("Média do Median Absolute Error (MedAE):", media_medae)
print("Média do R^2 dentro da amostra:", media_r2_in_sample)
print("Média do R^2 fora da amostra:", media_r2_out_of_sample)


Média do Median Absolute Error (MedAE): 1851483.3502209324
Média do R^2 dentro da amostra: 0.9477384869911916
Média do R^2 fora da amostra: 0.6651342673294043


SVR (NÃO LINEAR)

In [18]:
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.metrics import median_absolute_error, r2_score
import numpy as np

# Suponha que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_in_sample_scores = []
r2_out_of_sample_scores = []

# Treinar e avaliar o modelo usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo SVR com kernel RBF
    modelo_svr = SVR(kernel='poly', degree=3, C=1.0)

    modelo_svr.fit(X_train, y_train)
    y_pred = modelo_svr.predict(X_test)

    # Calcular o erro mediano absoluto (Median Absolute Error)
    medae = median_absolute_error(y_test, y_pred)
    medae_scores.append(medae)

    # Calcular o R^2 dentro da amostra
    y_train_pred = modelo_svr.predict(X_train)
    r2_in_sample = r2_score(y_train, y_train_pred)
    r2_in_sample_scores.append(r2_in_sample)

    # Calcular o R^2 fora da amostra
    r2_out_of_sample = r2_score(y_test, y_pred)
    r2_out_of_sample_scores.append(r2_out_of_sample)

# Média das métricas de avaliação em todas as dobras
media_medae = np.mean(medae_scores)
media_r2_in_sample = np.mean(r2_in_sample_scores)
media_r2_out_of_sample = np.mean(r2_out_of_sample_scores)

print("Média do Median Absolute Error (MedAE):", media_medae)
print("Média do R^2 dentro da amostra:", media_r2_in_sample)
print("Média do R^2 fora da amostra:", media_r2_out_of_sample)


Média do Median Absolute Error (MedAE): 1855809.6678745435
Média do R^2 dentro da amostra: -0.043430143845759515
Média do R^2 fora da amostra: -0.22719244055868618


CART_simples (NÃO LINEAR)

In [19]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Suponha que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
mae_scores = []
mse_scores = []
r2_in_sample_scores = []
r2_out_of_sample_scores = []

# Treinar e avaliar o modelo usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de árvore de decisão com os parâmetros desejados
    modelo_arvore = DecisionTreeRegressor(min_samples_split=5, random_state=42)

    # Criar o modelo Bagging com 500 árvores
    bagging_model = BaggingRegressor(base_estimator=modelo_arvore, n_estimators=500, bootstrap=True, random_state=42)

    bagging_model.fit(X_train, y_train)
    y_pred = bagging_model.predict(X_test)

    # Calcular as métricas de avaliação
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2_in_sample = bagging_model.score(X_train, y_train)
    r2_out_of_sample = bagging_model.score(X_test, y_test)

    mae_scores.append(mae)
    mse_scores.append(mse)
    r2_in_sample_scores.append(r2_in_sample)
    r2_out_of_sample_scores.append(r2_out_of_sample)

# Média das métricas de avaliação em todas as dobras
media_mae = np.mean(mae_scores)
media_mse = np.mean(mse_scores)
media_r2_in_sample = np.mean(r2_in_sample_scores)
media_r2_out_of_sample = np.mean(r2_out_of_sample_scores)

print("Média do Mean Absolute Error (MAE):", media_mae)
print("Média do Mean Squared Error (MSE):", media_mse)
print("Média do R^2 dentro da amostra:", media_r2_in_sample)
print("Média do R^2 fora da amostra:", media_r2_out_of_sample)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en

Média do Mean Absolute Error (MAE): 3831600.3528409107
Média do Mean Squared Error (MSE): 189012557056774.53
Média do R^2 dentro da amostra: 0.9513509764888994
Média do R^2 fora da amostra: 0.5030578777265446


FLORESTA ALEATÓRIA (NÃO LINEAR)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Certifique-se de que os tipos de dados sejam numéricos
X = X.astype(float)
y = y.astype(float)

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de Floresta Aleatória com os parâmetros desejados
    random_forest_model = RandomForestRegressor(n_estimators=500, random_state=42)
    #random_forest_model = RandomForestRegressor(n_estimators=500, max_depth=10, random_state=42)
    #random_forest_model = RandomForestRegressor(n_estimators=500, max_features=4, random_state=42)
    #random_forest_model = RandomForestRegressor(n_estimators=500, max_features=0.3, random_state=42)


    random_forest_model.fit(X_train, y_train)
    y_pred = random_forest_model.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

    # Visualizar a importância das variáveis
    importancia_variaveis = random_forest_model.feature_importances_
    print("Importância das Variáveis:", importancia_variaveis)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


Importância das Variáveis: [0.85198725 0.14801275]
Importância das Variáveis: [0.84385787 0.15614213]
Importância das Variáveis: [0.88367251 0.11632749]
Importância das Variáveis: [0.86212843 0.13787157]
Importância das Variáveis: [0.83930719 0.16069281]
Importância das Variáveis: [0.88525781 0.11474219]
Importância das Variáveis: [0.86254208 0.13745792]
Importância das Variáveis: [0.8366682 0.1633318]
Importância das Variáveis: [0.84572728 0.15427272]
Importância das Variáveis: [0.85194658 0.14805342]
Média do MedAE escalado pelo valor (ou preço) em percentagem: 14.43210797407156
Média do R²: 0.542486201674424


GRADIENT BOOSTING (NÃO LINEAR)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de Regressão Gradient Boosting com os parâmetros desejados
    gradient_boosting_model = GradientBoostingRegressor(n_estimators=500, random_state=42)

    gradient_boosting_model.fit(X_train, y_train)
    y_pred = gradient_boosting_model.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

    # Visualizar a importância das variáveis
    importancia_variaveis = gradient_boosting_model.feature_importances_
    print("Importância das Variáveis:", importancia_variaveis)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


Importância das Variáveis: [0.85935704 0.14064296]
Importância das Variáveis: [0.81805766 0.18194234]
Importância das Variáveis: [0.8754046 0.1245954]
Importância das Variáveis: [0.85055114 0.14944886]
Importância das Variáveis: [0.83842492 0.16157508]
Importância das Variáveis: [0.89175798 0.10824202]
Importância das Variáveis: [0.86364624 0.13635376]
Importância das Variáveis: [0.83163422 0.16836578]
Importância das Variáveis: [0.84431224 0.15568776]
Importância das Variáveis: [0.84499112 0.15500888]
Média do MedAE escalado pelo valor (ou preço) em percentagem: 12.699941787318881
Média do R²: 0.351145473964532


REDES NEURAIS (NÃO LINEAR)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de regressão de rede neural com os parâmetros desejados
    modelo_rede_neural = MLPRegressor(hidden_layer_sizes=(30), max_iter=1000, random_state=42)

    modelo_rede_neural.fit(X_train, y_train)
    y_pred = modelo_rede_neural.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perce

Média do MedAE escalado pelo valor (ou preço) em percentagem: 11.691986211955475
Média do R²: 0.6298537572510516
